<a href="https://colab.research.google.com/github/Satyake/Getting-Started-with-TF2/blob/main/Text_Generator_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def read_file(filepath):
  with open(filepath) as f:
    text=f.read()
    text=text[:100000]
  return text

In [ ]:
import spacy 
nlp=spacy.load('en')

In [ ]:
def seperate(doc):
  return [ token.text for token in nlp(doc) if token.text not in '\n\n \n\n\n!".#$<-@?~{}']

In [ ]:
directory=read_file('/content/Shakespeare.txt')

In [ ]:
words=seperate(directory)

In [ ]:
len(words)

21369

In [ ]:
#network to predict next word using 26 words

In [ ]:
train_len=30
text_sequences=[]
for i in range(train_len,len(words)):
  seq=words[i-train_len:i]
  text_sequences.append(seq)


In [ ]:
' '.join(text_sequences[0])

'First Citizen : Before we proceed any further , hear me speak All : Speak , speak First Citizen : You are all resolved rather to die than to famish'

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tknizer=Tokenizer()
tknizer.fit_on_texts(text_sequences) 
sequences=tknizer.texts_to_sequences(text_sequences)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
tknizer.index_word

In [ ]:
#tknizer.word_counts
vocabulary_size=len(tknizer.word_counts)

In [ ]:
vocabulary_size

3061

In [ ]:
import numpy as np 
import keras
import tensorflow as tf
sequences=np.array(sequences)

In [ ]:
sequences.shape
x=sequences[:,:-1]
y=sequences[:,-1]

In [ ]:
y.shape

(21339,)

In [ ]:
x.shape

(21339, 29)

In [ ]:
seq_length=x.shape[0]
seq_length

21339

In [ ]:
y=tf.keras.utils.to_categorical(y,num_classes=vocabulary_size+1)

In [ ]:
y.shape

(21339, 3062)

In [ ]:
vocabulary_size

3061

In [ ]:
seq_length=x.shape[1]

In [ ]:
seq_length

29

In [ ]:
from keras.models import Sequential

from keras.layers import Dense,LSTM,Embedding,Dropout,Bidirectional
import tensorflow as tf

In [ ]:
def model1():
 model=tf.keras.models.Sequential()
 model.add(tf.keras.layers.Embedding(vocabulary_size+1,32,input_length=seq_length))
 model.add(tf.keras.layers.LSTM(50,return_sequences=True))
 model.add(tf.keras.layers.LSTM(10,return_sequences=False))
 model.add(tf.keras.layers.Dense(vocabulary_size+1,activation='softmax'))
 model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
 return model

In [ ]:
model=model1()

In [ ]:
tknizer.word_index

In [ ]:
history=model.fit(x,y,epochs=150,batch_size=128)

In [213]:
model.predict(x)[0]

array([5.0164374e-14, 4.6653865e-13, 1.9111874e-12, ..., 4.8883879e-13,
       2.2235707e-09, 2.3891425e-11], dtype=float32)

In [232]:
def generate_text(model,tknizer,seed_text,num_gen_words, seq_len):
  output_text=[]
  for i in range(num_gen_words):
    encoded_text=tknizer.texts_to_sequences(seed_text)[0]
    padded=tf.keras.preprocessing.sequence.pad_sequences([encoded_text],maxlen=seq_len)
    predict_index=model.predict(padded)[0]
    predict_index=np.argmax(predict_index)
    predicted_word=tknizer.index_word[predict_index]
    seed_text+=' ' + predicted_word
    output_text.append(predicted_word)
  return ' '.join(output_text)


text='and throwing the clothes to one side he really did this'

generate_text(model,tknizer,text,2,30)


'pleasing pleasing'

In [ ]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
  output_text=[]
  input_text=seed_text 
  for i in range(num_gen_words):
    encoded_text=tknizer.texts_to_sequences([input_text])[0]
    pred_word_ind=model.predict(encoded_text)[0]
    pred_word_ind=np.argmax(pred_word_ind)
    pred_word=tokenizer.index_word[pred_word_ind]
    input_text+=' ' + pred_word
    output_text.append(pred_word)

  return ' '.join()
    

In [225]:
text='and throwing the clothes to one side he really did this'


In [226]:
generate_text(model,tknizer,29,text,25)

TypeError: ignored